In [ ]:
from datasets import load_dataset, load_metric, load_from_disk
from datasets import ClassLabel
import random
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
from transformers import Trainer


#model_fp = "/srv/scratch/z5363024/thesis/timit_test"
model_fp = "D:/Users/andrew/uni_work/thesis/code/timit_test"
finetuned_results_fp = model_fp + 'finetuned_results.csv'
dataset_fp = 'D:/Users/andrew/uni_work/thesis/code/timit_test/timit'

timit = load_dataset("timit_asr")  # Load the dataset from Hugging Face
#timit.save_to_disk(dataset_fp)
#timit = load_from_disk(dataset_fp)

#timit = load_dataset("arrow", data_files={
#    "train": [dataset_fp + '/train/data-00000-of-00005.arrow', dataset_fp + '/train/data-00001-of-00005.arrow', 
#    dataset_fp + '/train/data-00002-of-00005.arrow', dataset_fp + '/train/data-00003-of-00005.arrow', 
#    dataset_fp + '/train/data-00004-of-00005.arrow'], 
#    "test": [dataset_fp + '/test/data-00000-of-00002.arrow', dataset_fp + '/test/data-00001-of-00002.arrow']}
#    )

#timit_train = load_from_disk(dataset_fp + '_mostafa/train')


timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])


Reusing dataset timit_asr (C:\Users\andre\.cache\huggingface\datasets\timit_asr\clean\2.0.1\b11b576ddcccbcefa7c9f0c4e6c2a43756f3033adffe0fb686aa61043d0450ad)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
# display random samples
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(timit["train"].remove_columns(["audio", "file"]), num_examples=10)

,text
0,We now generalize these ideas for general binomial experiments.
1,The cow wandered from the farmland and became lost.
2,Replace it with the statue of one or another of the world's famous dictators.
3,She had your dark suit in greasy wash water all year.
4,"When all else fails, use force."
5,Don't ask me to carry an oily rag like that.
6,Etiquette mandates compliance with existing regulations.
7,She had your dark suit in greasy wash water all year.
8,"It gave her a lewd, winking effect."
9,Don't ask me to carry an oily rag like that.


In [3]:
# remove special characters and make lowercase
import re

def remove_special_characters(batch):
    chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

timit = timit.map(remove_special_characters)
show_random_elements(timit["train"].remove_columns(["audio", "file"])) # check text

Parameter 'function'=<function remove_special_characters at 0x000001555826BB80> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

,text
0,in simpler terms it amounts to pointing the platform in the proper direction
1,a moth zigzagged along the path through otto's garden
2,the thick elm forest was nearly overwhelmed by dutch elm disease
3,we could barely see the fjords through the snow flurries
4,todd placed top priority on getting his bike fixed
5,the full moon shone brightly that night
6,they all like long hot showers
7,being based on so few events these results are of dubious validity
8,hey come back he shouted
9,at intervals an alumni directory is issued


In [4]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [5]:
# not really sure if this stuff is necessary or was shown for learning
vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [6]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [7]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'l': 0,
 'n': 1,
 "'": 2,
 'c': 3,
 'h': 4,
 'p': 5,
 'u': 6,
 'd': 7,
 'a': 8,
 'g': 9,
 'v': 10,
 'j': 11,
 'y': 12,
 ' ': 13,
 'x': 14,
 'w': 15,
 'o': 16,
 'k': 17,
 'm': 18,
 'i': 19,
 'e': 20,
 's': 21,
 'b': 22,
 'f': 23,
 'q': 24,
 'z': 25,
 't': 26,
 'r': 27}

In [8]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [9]:

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

30

In [10]:
# not sure if this stuff will work
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)


In [11]:

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


In [12]:

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)


In [13]:

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [14]:
timit["train"][0]["file"]

'C:\\Users\\andre\\.cache\\huggingface\\datasets\\downloads\\extracted\\c19a2fc53975ac1b608d4cad910e3c20adfdf1687e74862fe790181ebf3b943a\\data\\TRAIN\\DR4\\MMDM0\\SI681.WAV'

In [15]:
timit["train"][0]["audio"]

{'path': 'C:\\Users\\andre\\.cache\\huggingface\\datasets\\downloads\\extracted\\c19a2fc53975ac1b608d4cad910e3c20adfdf1687e74862fe790181ebf3b943a\\data\\TRAIN\\DR4\\MMDM0\\SI681.WAV',
 'array': array([-2.1362305e-04,  6.1035156e-05,  3.0517578e-05, ...,
        -3.0517578e-05, -9.1552734e-05, -6.1035156e-05], dtype=float32),
 'sampling_rate': 16000}

In [16]:
# checking audio
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(timit["train"]))

print(timit["train"][rand_int]["text"])
ipd.Audio(data=np.asarray(timit["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)

the boston ballet overcame their funding shortage 


In [17]:
# checking sample
rand_int = random.randint(0, len(timit["train"]))

print("Target text:", timit["train"][rand_int]["text"])
print("Input array shape:", np.asarray(timit["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", timit["train"][rand_int]["audio"]["sampling_rate"])

Target text: regular attendance is seldom required 
Input array shape: (33895,)
Sampling rate: 16000


In [18]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch


In [19]:

timit = timit.map(prepare_dataset, remove_columns=timit.column_names["train"])#, num_proc=4)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [20]:
# everything above 4 seconds is filtered out to minimise memory usage
max_input_length_in_sec = 4.0
timit["train"] = timit["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

  0%|          | 0/5 [00:00<?, ?ba/s]

In [21]:
# seems to be the way padding and masking is done
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [22]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [23]:
wer_metric = load_metric("wer")

In [24]:

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [25]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)


d:\Users\andrew\anaconda3\lib\site-packages\transformers\configuration_utils.py:356: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
d:\Users\andrew\anaconda3\lib\site-packages\transformers\modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer 

In [26]:
model.freeze_feature_encoder()

In [27]:
# is said to be good for timit
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=model_fp,
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=30,
  #fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)


In [28]:

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit["train"],
    eval_dataset=timit["test"],
    tokenizer=processor.feature_extractor,
)

In [29]:
### training

trainer.train()


The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
d:\Users\andrew\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3978
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 14940


  0%|          | 0/14940 [00:00<?, ?it/s]

d:\Users\andrew\anaconda3\lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1680
  Batch size = 8


{'loss': 3.5057, 'learning_rate': 5e-05, 'epoch': 1.0}


  0%|          | 0/210 [00:00<?, ?it/s]

...

  0%|          | 0/210 [00:00<?, ?it/s]

Saving model checkpoint to D:/Users/andrew/uni_work/thesis/code/timit_test\checkpoint-10000
Configuration saved in D:/Users/andrew/uni_work/thesis/code/timit_test\checkpoint-10000\config.json


{'eval_loss': 0.5319012403488159, 'eval_wer': 0.3535938253738543, 'eval_runtime': 473.1091, 'eval_samples_per_second': 3.551, 'eval_steps_per_second': 0.444, 'epoch': 20.08}


Model weights saved in D:/Users/andrew/uni_work/thesis/code/timit_test\checkpoint-10000\pytorch_model.bin
Feature extractor saved in D:/Users/andrew/uni_work/thesis/code/timit_test\checkpoint-10000\preprocessor_config.json
Deleting older checkpoint [D:\Users\andrew\uni_work\thesis\code\timit_test\checkpoint-9000] due to args.save_total_limit
d:\Users\andrew\anaconda3\lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


KeyboardInterrupt: 

In [ ]:
model.save_pretrained(model_fp)
processor = Wav2Vec2Processor.from_pretrained(model_fp)
model = Wav2Vec2ForCTC.from_pretrained(model_fp).cuda()

In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [ ]:

results = timit["test"].map(map_to_result, remove_columns=timit["test"].column_names)
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

show_random_elements(results) # checking

#model.to("cuda")

#with torch.no_grad():
#  logits = model(torch.tensor(timit["test"][:1]["input_values"], device="cuda")).logits

#pred_ids = torch.argmax(logits, dim=-1)